### Extract text from Pdf documents and apply NER

In [46]:
import pdfplumber, os
import os.path as osp
import pandas as pd
from pathlib import Path
from pdf_extract.services import file
from pdf_extract.config import global_config as glob
from pdf_extract.config import config
from importlib import reload
import fasttext

reload(glob)
reload(config)

<module 'pdf_extract.config.config' from '/home/alexv84/Documents/GitHub/pdf_extract/src/pdf_extract/config/config.py'>

In [10]:
list_of_NEG_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'negatives'))
n_neg = len(list_of_NEG_docs)

list_of_POS_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'positives'))
n_pos = len(list_of_POS_docs)

print(n_pos)
print(n_neg)


12
13


In [11]:
# fname = 'CV4'

# i, page_objects, text = 0, {}, []
# # The open method returns an instance of the pdfplumber.PDF class.
# with pdfplumber.open(osp.join(glob.UC_DATA_DIR, f"example_cvs/{fname}.pdf")) as pdf:
#     while i < len(pdf.pages):
#         page = pdf.pages[i]
#         page_objects[str(i+1)] = page.extract_text(x_tolerance=1).split('\n')
#         #mystring = page_objects[str(i+1)].replace('\r', '').replace('\n', '')
#         text += page_objects[str(i+1)]
#         print(f"Page {i}")
#         #print(page.extract_text())
#         i += 1


In [12]:
#page_objects['1']

Read in all Pdfs in directory:

Postives only:

In [32]:
raw_corpus1 = pd.DataFrame(columns=['text', 'label'])

for z, fname in enumerate(list_of_POS_docs):
    #print(fname)
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"positives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus1.loc[z] = text  
raw_corpus1['label'] = 'positive'

Negatives only:

In [33]:
raw_corpus2 = pd.DataFrame(columns=['text','label'])

for z, fname in enumerate(list_of_NEG_docs):
    #print(fname)
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"negatives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus2.loc[z] = text   
raw_corpus2['label'] = 'negative'

In [34]:
raw_corpus = pd.concat([raw_corpus1,raw_corpus2], axis=0, ignore_index=True)
#raw_corpus

In [35]:
#page_objects

#page.extract_words()

In [36]:
page_instance = page_objects['1']

In [37]:
#pdf.metadata
#page_instance

Preprocess text corpus:

In [38]:
from pdf_extract.resources import preprocessor as preproc

reload(preproc)

# Preprocess corpus:
cleaner = preproc.clean_text(language='english', lemma = False, stem = False)

X = raw_corpus['text']

X_cl = cleaner.fit_transform(X)

#docs = X_cl.tolist()    

Using english language.
Using 179 stop words.
Added 0 stopword(s).
Adding custom stop words...
Setting to lower cases.
Removing whitespaces.
Applying word tokenizer.
Removing custom stopwords.
Removing punctuations.
Removing numbers.
Removing digits.
Removing non-alphabetic characters.
Removing short tokens.
Finished preprocessing!


[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
combine_fct = lambda x: '__label__{}'.format(x.label)

raw_corpus['text_cleaned'] = X_cl
raw_corpus['fasttext_label'] = raw_corpus.apply(combine_fct, axis = 1)

X = raw_corpus[['fasttext_label', 'text_cleaned']].reset_index(drop=True)
X.head()

,fasttext_label,text_cleaned
0,__label__positive,ghassen makhlouf student computer science link...
1,__label__positive,tobias watzel zusammenfassung experte machine ...
2,__label__positive,gabriel kordon kordon mangfallstraße rosenheim...
3,__label__positive,ceyda akbulut phone email akbulut gmail linked...
4,__label__positive,boyao zhang statistician data scientist boyao ...


In [35]:
#X

In [34]:
#X_cl

In [45]:
reload(file)

config_output = config.io['output']
Path(glob.UC_DATA_DIR).mkdir(parents=True, exist_ok=True)

file.TXTService(verbose=True, root_path=glob.UC_DATA_DIR, **config_output['service']['TXTService']).doWrite(X)

TXT Service output to file: /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/train_data_fasttext.txt


In [50]:
# Train model
#-------------
#reload(utils)

mtype = 'skipgram'    #'cbow'  

print(f"-- Using {'Skip-gram' if mtype == 'skipgram' else 'CBOW'} --")
# Training word embeddings: 
model = fasttext.train_supervised(input = os.path.join(glob.UC_DATA_DIR,'train_data_fasttext.txt'), wordNgrams = 2, dim=100, epoch=60, lr=0.001, verbose=2)
print("Done!")

-- Using Skip-gram --


Read 0M words
Number of words:  3095
Number of labels: 2


Done!


Progress: 100.0% words/sec/thread:  406609 lr:  0.000000 avg.loss:  0.693146 ETA:   0h 0m 0s


In [34]:
#page_instance.pdf.to_dict()

In [20]:
#page_instance.extract_words()

### NER:

In [37]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_lg")

Use pretrained language identification:

In [60]:
import fasttext

model = fasttext.load_model(osp.join(glob.UC_LANG_ID, 'lid.176.bin'))

label, score = model.predict("Hallo ich studiere Mathe in München", k=2)  # top 2 matching languages
label[0]


'__label__de'

In [38]:
i = 1
#raw_text = X.loc[i]
raw_text = X_cl.loc[i]
#raw_text

In [37]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."

In [39]:
text1 = NER(raw_text)

In [40]:
# for word in text1.ents:
#     print(word.text,word.label_)

In [41]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

In [42]:
spacy.explain("GPE")

'Countries, cities, states'

In [44]:
#displacy.render(text1,style="ent",jupyter=True)

In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [28]:
URL="https://www.zeebiz.com/markets/currency/news-cryptocurrency-news-today-june-12-bitcoin-dogecoin-shiba-inu-and-other-top-coins-prices-and-all-latest-updates-158490"

In [29]:
html_content = requests.get(URL).text

In [30]:
soup = BeautifulSoup(html_content, "lxml")

In [31]:
body=soup.body.text

In [32]:
body= body.replace('\n', ' ')
#body= body.replace('t', ' ')
#body= body.replace('r', ' ')
#body= body.replace('xa0', ' ')
#body=re.sub(r'[^ws]', '', body)

In [148]:
#body

In [149]:
text3= NER(body)